## 17. 인수를 순회할 때는 방어적으로 하자
* **파라미터로 객체의 리스트를 받는 함수에서 리스트를 여러 번 순회해야 할 때**
    - 예: 입력을 합산해 총합을 구하고, 개개의 값을 총합으로 나누어 비중을 알아내는 정규화 함수

In [2]:
# 정규화 함수
def normalize(numbers):
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

In [3]:
# 리스트를 받아 동작
visits = [15, 35, 80]
percentages = normalize(visits)
print(percentages)

[11.538461538461538, 26.923076923076923, 61.53846153846154]


* **제너레이터** 정의하기 (**Better Way 16** 참조)
    - 나중에 같은 함수를 재사용해 더 큰 데이터셋에 적용 가능

In [10]:
# 데이터 파일 생성
path = 'my_numbers.txt'
with open(path, 'w') as f:
    for i in (15, 35, 80):
        f.write('%d\n' % i)

In [11]:
# 파일에서 데이터를 읽어 오는 제너레이터 정의
def read_visits(data_path):
    with open(data_path) as f:
        for line in f:
            yield int(line)

* **제너레이터의 반환 값에 normalize를 호출하면 아무 결과도 생성되지 않음!**

In [12]:
# 제너레이터 반환 값에 normalize 호출
it = read_visits('my_numbers.txt')
percentages = normalize(it)
print(percentages)

[]


* **이터레이터가 결과를 한 번만 생성하기 때문**
* **이미 StopIteration 예외를 일으킨 이터레이터/제너레이터를 순회하면 어떤 결과도 얻을 수 없음**

In [8]:
# 순회한 다음 다시 순회하기
it = read_visits('my_numbers.txt')
print(list(it))
print(list(it)) # 이미 소진함

[15, 35, 80]
[]


* **이미 소진한 이터레이터를 순회하더라도 오류가 일어나지 않음**
* for 루프, list 생성자, 파이썬 표준 라이브러리의 많은 함수는 정상적 동작 과정에서 StopIteration 예외가 일어날 것이라 기대
    - 이런 함수는 **결과가 없는 이터레이터** 와 **결과가 있었지만 이미 소진한 이터레이터**의 차이를 알려주지 않음

* **해결 방법: 입력 이터레이터를 방어적으로 복사**
    - 입력 이터레이터를 명시적으로 소진
    - 전체 콘텐츠의 복사본을 리스트에 저장
    - 이후 리스트 버전의 데이터를 필요한 만큼 순회

In [9]:
# 입력 데이터를 방어적으로 복사하는 정규화 함수
def normalize_copy(numbers):
    numbers = list(numbers) # 이터레이터를 복사함
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

In [13]:
# 제너레이터의 반환 값에도 올바르게 동작함
it = read_visits('my_numbers.txt')
percentages = normalize_copy(it)
print(percentages)

[11.538461538461538, 26.923076923076923, 61.53846153846154]


* 위 방법의 문제점
    - 입력받은 이터레이터 콘텐츠의 복사본이 클 수 있음
    - 큰 이터레이터를 복사하면 프로그램의 메모리가 고갈되어 동작을 멈출 수 있음
* 해결 방법
    - 호출될 때마다 새 이터레이터를 반환하는 함수를 받게 하기
        - 방법 1: 매번 새 이터레이터를 생성하는 람다 표현식 넘겨주기

In [15]:
# 호출될 때마다 새 이터레이터를 반환하는 함수를 받음
def normalize_func(get_iter):
    total = sum(get_iter()) # 새 이터레이터
    result = []
    for value in get_iter(): # 새 이터레이터
        percent = 100 * value / total
        result.append(result)
    return result

In [18]:
# 제너레이터를 호출해서 매번 새 이터레이터를 생성하는 람다 표현식 넘겨주기
percentages = normalize_func(lambda: read_visits(path))

* 해결 방법
    - 호출될 때마다 새 이터레이터를 반환하는 함수를 받게 하기
        - 방법 2 (더 세련됨!): 이터레이터 프로토콜(iterator protocol)을 구현한 새 컨테이너 클래스 제공

* **이터레이터 프로토콜**
    - 파이썬의 for 루프 및 관련 표현식이 컨테이너 타입의 콘텐츠를 탐색하는 방법을 나타냄
        - 파이썬은 for x in foo 같은 문장을 만나면 실제로는 iter(foo)를 호출
        - 내장 함수 iter는 특별한 메서드인 foo.\__iter\__를 호출
        - \__iter\__ 메서드는 (\__next\__라는 특별한 메서드를 구현하는) 이터레이터 객체를 반환해야 함
        - for 루프는 이터레이터를 모두 소진할 때까지 (i.e., StopIteration 예외 발생시까지) 이터레이터 객체에 내장 함수 next를 계속 호출

* 클래스의 \__iter\__ 메서드를 제너레이터로 구현하면 이렇게 동작하게 만들 수 있음

In [20]:
# 데이터 파일을 읽는 이터러블(순회 가능) 컨테이너 클래스
class ReadVisits(object):
    def __init__(self, data_path):
        self.data_path = data_path
    
    def __iter__(self):
        with open(self.data_path) as f:
            for line in f:
                yield int(line)

In [21]:
# 원래 함수를 수정하지 않고 넘겨도 제대로 동작함
visits = ReadVisits(path)
percentages = normalize(visits)
print(percentages)

[11.538461538461538, 26.923076923076923, 61.53846153846154]


* 제대로 동작하는 이유
    - normalize의 sum 메서드가 새 이터레이터 객체를 할당하려고 ReadVisits.\__iter\__를 호출하기 때문
    - 숫자를 정규화하는 for 루프도 두 번째 이터레이터 객체를 할당할 때 \__iter\__를 호출
    - 두 이터레이터는 독립적으로 동작
        - 각각의 순회 과정에서 모든 입력 데이터 값을 얻을 수 있음
    - 단점: 입력 데이터를 여러 번 읽음

* 파라미터가 단순한 이터레이터가 아님을 보장하는 함수가 필요
    - 내장 함수 iter
        - 이터레이터를 넘기면 이터레이터 자체가 반환됨
        - 컨테이너 타입을 넘기면 매번 새 이터레이터 객체가 반환됨
    - 입력값을 테스트해서 이터레이터면 TypeError를 일으켜 거부하게 만듦

In [24]:
# 방어적 정규화 함수
def normalize_defensive(numbers):
    if iter(numbers) is iter(numbers):
        raise TypeError('Must supply a container') #이터레이터 — 거부!
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

* 입력 이터레이터 전체를 복사하고 싶지 않지만, 입력 데이터를 여러 번 순회해야 할 떄 사용하면 좋음
* 이터레이터 프로토콜을 따르는 컨테이너 타입을 입력으로 받으면 기대대로 동작

In [26]:
visits = [15, 35, 80]
normalize_defensive(visits) # 오류 없음

[11.538461538461538, 26.923076923076923, 61.53846153846154]

In [27]:
visits = ReadVisits(path)
normalize_defensive(visits) # 오류 없음

[11.538461538461538, 26.923076923076923, 61.53846153846154]

* 입력이 이터러블이어도 컨테이너가 아니면 예외 발생

In [28]:
it = iter(visits)
normalize_defensive(it)

TypeError: Must supply a container

## 18. 가변 위치 변수로 깔끔하게 보이게 하자

* **선택적인 위치 인수(\*args = star args) 사용**
    - 함수 호출을 더 명확하게 할 수 있고 보기에 방해가 되는 요소를 없앨 수 있음

In [29]:
# 예: 디버그 정보 로그로 남기기
# 인수의 개수가 고정되어 있을 때는 메시지, 값 리스트를 받는 함수 필요
def log(message, values):
    if not values:
        print(message)
    else:
        values_str = ', '.join(str(x) for x in values)
        print('%s: %s' % (message, values_str))

In [30]:
log('My numbers are', [1, 2])
log('Hi there', [])

My numbers are: 1, 2
Hi there


* 넘길 값이 없을 때 빈 리스트를 넘기는 것은 번거로움
    - 두 번째 인수를 아예 남겨둔다면 좋음
    - 파이썬의 경우 \* 기호를 마지막 위치 파라미터 이름 앞에 붙이면 됨
    - 첫 번째 파라미터는 필수
    - 다음에 나오는 위치 인수는 몇 개든 선택적

In [34]:
# 함수 본문은 수정할 필요 없음
# 호출하는 부분만 수정
def log(message, *values): # 유일하게 다른 부분
    if not values:
        print(message)
    else:
        values_str = ', '.join(str(x) for x in values)
        print('%s: %s' % (message, values_str))

In [35]:
log('My numbers are', 1, 2)
log('Hi there') # 훨씬 나음

My numbers are: 1, 2
Hi there


* **\* 연산자를 써서 리스트를 가변 인수 함수를 호출하는 데 사용 가능**
    - 시퀀스에 들어 있는 아이템들을 위치 인수로 전달함

In [36]:
favorites = [7, 33, 99]
log('Favorite colors', *favorites)

Favorite colors: 7, 33, 99


* 가변 개수의 위치 인수를 받는 방법: 2가지 문제가 있음
    - **(1) 가변 인수가 함수에 전달되기 전 항상 튜플로 변환됨**
        - 함수를 호출하는 쪽에서 제너레이터에 \* 연산자를 쓰면 제너레이터가 모두 소진될 때까지 순회됨
        - 결과 튜플은 제너레이터가 생성한 모든 값을 담으므로 메모리를 많이 차지할 수 있음

In [38]:
def my_generator():
    for i in range(10):
        yield i

def my_func(*args):
    print(args)

In [39]:
it = my_generator()
my_func(*it)

(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)


* **\*args를 받는 함수는 인수 리스트에 있는 입력의 수가 적당히 적다는 사실을 알 때 좋은 방법**
    - 많은 리터럴이나 변수 이름을 한꺼번에 넘기는 함수 호출에 이상적
    - 편의와 코드 가독성을 높이기 위해 사용

* 가변 개수의 위치 인수를 받는 방법: 2가지 문제가 있음
    - **(2) 추후에 호출 코드를 모두 변경하지 않고서는 새 위치 인수를 추가할 수 없음**
        - 인수 리스트의 앞쪽에 위치 인수를 추가하는 경우
            - 기존 호출 코드가 수정 없이는 잘못 동작

In [42]:
# 인수 리스트 앞쪽에 위치 인수 추가
def log(sequence, message, *values):
    if not values:
        print('%s %s' % (sequence, message))
    else:
        values_str = ', '.join(str(x) for x in values)
        print('%s: %s: %s' % (sequence, message, values_str))

In [43]:
log(1, 'Favorites', 7, 33) # 새로운 용법은 OK
log('Favorite numbers', 7, 33) # 오래된 용법은 제대로 동작하지 않음

1: Favorites: 7, 33
Favorite numbers: 7: 33


* 두 번째 호출이 sequence 인수를 받지 못했기 때문에 7을 message 파라미터로 사용함
    - 코드에서 예외를 일으키지 않고 계속 실행되는 버그이므로 발견이 어려움
    - 이런 문제가 생길 가능성을 완전히 없애는 법
        - \*args를 받는 함수를 확장할 때 **키워드 전용(keyward-only) 인수**를 사용(**Better Way 21** 참조)

## 19. 키워드 인수로 선택적인 동작을 제공하자

* 함수를 호출할 때 인수를 위치로 전달할 수 있음

In [44]:
# 나머지 함수
def remainder(number, divisor):
    return number % divisor

In [45]:
assert remainder(20, 7) == 6

* 파이썬 함수의 위치 인수를 모두 키워드로 전달할 수도 있음
* 인수의 이름을 함수 호출 괄호 안의 할당문에서 사용
* 필요한 위치 인수를 모두 지정하면 키워드 인수로도 전달 가능
* 키워드와 위치 인수를 섞어서 사용할 수 있음

In [46]:
# 모두 동일한 호출
remainder(20, 7)
remainder(20, divisor=7)
remainder(number=20, divisor=7)
remainder(divisor=7, number=20)

6

* 위치 인수는 키워드 인수 앞에 지정해야 함

In [47]:
# 키워드 인수를 위치 인수 앞에 지정하면 오류 발생
remainder(number=20, 7)

SyntaxError: positional argument follows keyword argument (<ipython-input-47-aae2da038bb6>, line 2)

* 각 인수는 한 번만 지정할 수 있음

In [48]:
# 같은 인수를 여러 번 지정하면 오류 발생
remainder(20, number=7)

TypeError: remainder() got multiple values for argument 'number'

* **키워드 인수의 유연성: 3가지 이점**
    - **(1) 함수 호출을 더 명확하게 이해할 수 있음**
        - 메서드 구현을 보지 않고도 각각의 목적으로 어떤 파라미터를 사용했는지 바로 알 수 있음
        - 예: remainder(20, 7) vs. remainder(number=20, divisor=7)

* **키워드 인수의 유연성: 3가지 이점**
    - **(2) 함수 정의시 기본값을 설정할 수 있음**
        - 기본값을 사용하다 필요시 부가 기능을 제공할 수 있음
        - 반복 코드가 줄고 코드가 깔끔해짐

In [49]:
# 큰 통에 들어가는 액체의 유속 계산
def flow_rate(weight_diff, time_diff):
    return weight_diff / time_diff

In [50]:
weight_diff = 0.5
time_diff = 3
flow = flow_rate(weight_diff, time_diff)
print('%.3f kg per second' % flow)

0.167 kg per second


In [51]:
# 인수에 기간 환산 계수 추가
# 초당 킬로그램 단위가 아니라 다른 시간 단위로 계산하는 경우를 위함
def flow_rate(weight_diff, time_diff, period):
    return (weight_diff / time_diff) * period

In [52]:
# 이렇게 하면 함수를 호출할 때마다 일반적 경우(period ==1)에도 period 설정해야 함
flow_per_second = flow_rate(weight_diff, time_diff, 1)

In [53]:
# period 인수에 기본값 설정
def flow_rate(weight_diff, time_diff, period=1):
    return (weight_diff / time_diff) * period

In [54]:
# period가 선택적 인수가 됨
flow_per_second = flow_rate(weight_diff, time_diff)
flow_per_second = flow_rate(weight_diff, time_diff, period=3600)

* 위 코드는 간단한 기본값에는 잘 동작함
    - 기본값이 복잡할 때는 다루기 까다로움
        - None과 docstring을 사용한 동적 기본 인수 지정(**Better Way 20** 참조)

* **키워드 인수의 유연성: 3가지 이점**
    - **(3) 기존 호출 코드와 호환성을 유지하면서 함수 파라미터를 확장하는 강력한 수단**
        - 코드를 많이 수정하지 않고도 추가적 기능 제공 가능
        - 버그 가능성을 줄임

In [58]:
# 다른 무게 단위로도 유속 계산하기 위해 함수 확장
# units_per_kg 인수: 기본값 1, 반환 무게 단위 kg
# 기존 호출 코드 동작에는 변화 없음
def flow_rate(weight_diff, time_diff, 
              period=1, units_per_kg=1):
    return ((weight_diff / units_per_kg) / time_diff) * period

In [59]:
# 기존 호출 코드 동작에는 변화 없음
flow_per_second = flow_rate(weight_diff, time_diff, 1)

In [60]:
pounds_per_hour = flow_rate(weight_diff, time_diff,
                            period=3600, units_per_kg=2.2)

* 남아 있는 문제점
    - 선택적 키워드 인수를 여전히 위치 인수로도 넘길 수 있음
        - 대응하는 인수가 명확하지 않아 혼동을 일으킬 수 있음
        - **가장 좋은 방법**
            - **항상 키워드 이름으로 선택적 인수 지정**
            - **위치 인수로는 아예 넘기지 않기**

In [61]:
pounds_per_hour = flow_rate(weight_diff, time_diff, 3600, 2.2)

* 선택적 키워드 인수 사용시 \*args를 인수로 받는 함수에서 하위 호환성을 지키기 어려움(**Better Way 18** 참조)
* 더 좋은 방법은 **키워드 전용 인수**를 사용하는 것(**Better Way 21** 참조)